In [45]:
import pandas as pd
import os

In [46]:
species_list = pd.read_csv("/home/jacob/Desktop/EngineersForExploration/Data/Relevant BirdCLEF info/bird_classes.csv")

In [47]:
audio_path = "/home/jacob/Desktop/EngineersForExploration/Data/Relevant BirdCLEF info/audio/"
label_path = "/home/jacob/Desktop/EngineersForExploration/Data/Relevant BirdCLEF info/gt/"

In [49]:
# initializing the output dataframe that will contain all of the labels and the relevant metadata
# across each audio clip in a dataset. Should be in the format to work with it in the Python package
manual_df = pd.DataFrame()
# the ground truth labels lack column names, so I am filling them in closer to the end product
column_names = ["OFFSET","MANUAL ID"]
for clip_annotations in os.listdir(label_path):
    if clip_annotations.startswith(".~lock."):
        continue
    print(clip_annotations)
    # isolating the name of the clip from the csv file
    # will be used to extract the metadata from the equivelant wav file
    x = clip_annotations.split('.')
    clip_name = x[0]
    # taking in the labels for the audio clip
    clip_df = pd.read_csv(label_path+clip_annotations,names=column_names)
    # removing the annotations that occur over the same interval in the clip
    # first step in converting multi-class classifier into binary classifier.
    clip_df = clip_df.drop_duplicates(subset = ["OFFSET"])
    # second step to converting multi-class classifier to binary classifier
    # Isn't all that necessary since we don't use the MANUAL ID Column that much yet
    clip_df["MANUAL ID"] = "bird"
    # splitting the time into OFFSET and DURATION
    new = clip_df["OFFSET"].str.split("-", n = 1, expand = True)
    clip_df["OFFSET"] = new[0]
    clip_df["DURATION"] = 5
    #print(clip_df)
    # converting hours minutes seconds format into seconds
    new = clip_df["OFFSET"].str.split(":", n = 2, expand = True)
    seconds_offset = pd.to_numeric(new[0])*3600 + pd.to_numeric(new[1])*60 + pd.to_numeric(new[2])
    # resetting the offset column to be the proper offset format with respect to seconds in the clip
    clip_df["OFFSET"] = seconds_offset
    
    clip_df["FOLDER"] = audio_path
    clip_name = clip_name + ".wav"
    clip_df["IN FILE"] = clip_name
    clip_df["SAMPLE RATE"] = 32000
    clip_df["CHANNEL"] = 0
    clip_df["CLIP LENGTH"] = 600
    
    if manual_df.empty:
        manual_df = clip_df
    else:
        manual_df = manual_df.append(clip_df)
manual_df = manual_df.reindex(columns = ["FOLDER", "IN FILE", "CHANNEL", "CLIP LENGTH", "SAMPLE RATE", "OFFSET", "DURATION","MANUAL ID"])
manual_df.reset_index()
manual_df
manual_df.to_csv("BirdCLEF2020_Validation.csv",index=False)

SSW51_20170819.csv
SSW54_20170610.csv
PER54_20190116.csv
PER49_20190131.csv
PER53_20190116.csv
SSW50_20170819.csv
SSW49_20170520.csv
PER51_20190116.csv
PER50_20190116.csv
PER52_20190116.csv
SSW52_20170429.csv
SSW53_20170513.csv
